I need a Data set like this:

In [28]:
'''
   Yield  Property1  Property2  ParentMaterial1  ParentMaterial2
0     10          3          5                2                4
1     12          2          6                3                5
2     15          4          7                2                4
3     11          2          5                3                5
4     13          3          6                2                4
'''

'\n   Yield  Property1  Property2  ParentMaterial1  ParentMaterial2\n0     10          3          5                2                4\n1     12          2          6                3                5\n2     15          4          7                2                4\n3     11          2          5                3                5\n4     13          3          6                2                4\n'

Property 1 and Property2 will be features of output batches. ParentMaterial1 and ParentMaterial2 will be features of input batches.

In [29]:
import pandas as pd
import numpy as np

#data folder path definition
data_path='c:\\Users\\seyma\\personal_work\\sanofi\\Sanofi_Challenge\\Data\\'

#first load the excel document
Excel_file=pd.ExcelFile(data_path+"Datainput.xlsx")

#create data frames for each sheet in the excel file
Combined_Yields=pd.read_excel(Excel_file,sheet_name="Combined_Yields")
QC_Data=pd.read_excel(Excel_file,sheet_name="QC_Data")
IoT=pd.read_excel(Excel_file,sheet_name="IoT")
Campaign_Info=pd.read_excel(Excel_file,sheet_name="Campaign_Info")
Batch_Genealogy=pd.read_excel(Excel_file,sheet_name="Batch_Genealogy")

1- Create new_id Column, fix date format

In [30]:
# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in QC_Data
QC_Data['new_id'] = QC_Data['BATCH_ID'].astype(str) + QC_Data['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in Combined_Yields_full
Combined_Yields['new_id'] = Combined_Yields['BATCH_ID'].astype(str) + Combined_Yields['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in Batch_Genealogy
Batch_Genealogy['new_id'] = Batch_Genealogy['BATCH_ID'].astype(str) + Batch_Genealogy['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in Campaign_Info
Campaign_Info['new_id'] = Campaign_Info['BATCH_ID'].astype(str) + Campaign_Info['MATERIAL_ID']

# Merge the BATCH_ID and MATERIAL_ID columns into a new_id column in IoT
IoT['new_id'] = IoT['BATCH_ID'].astype(str) + IoT['MATERIAL_ID']


QC_Data['MANUFACTURE_DATE'] = QC_Data['MANUFACTURE_DATE'].replace(0, np.nan)
Combined_Yields['MANUFACTURE_DATE'] = Combined_Yields['MANUFACTURE_DATE'].replace(0, np.nan)
Batch_Genealogy['MANUFACTURE_DATE'] = Batch_Genealogy['MANUFACTURE_DATE'].replace(0, np.nan)

Combined_Yields['MANUFACTURE_DATE'] = pd.to_datetime(Combined_Yields['MANUFACTURE_DATE'], format='%Y%m%d')
QC_Data['MANUFACTURE_DATE'] = pd.to_datetime(QC_Data['MANUFACTURE_DATE'], format='%Y%m%d')
Batch_Genealogy['MANUFACTURE_DATE'] = pd.to_datetime(Batch_Genealogy['MANUFACTURE_DATE'], format='%Y%m%d')



2- Drop Unneccesary Colmuns and Rows and Duplicates

In [31]:
#drop na values in date column
Combined_Yields.dropna(subset=['MANUFACTURE_DATE'], inplace=True)
QC_Data.dropna(subset=['MANUFACTURE_DATE'], inplace=True)
Batch_Genealogy.dropna(subset=['MANUFACTURE_DATE'], inplace=True)

Combined_Yields_full=Combined_Yields[Combined_Yields.YIELD_NAME=="activity_coefficient"].dropna()
Combined_Yields_full=Combined_Yields_full.drop(['BATCH_ID','MATERIAL_ID','STAGE','YIELD_NAME'], axis=1)

# Delete rows where MOVEMENT_TYPE is equal to 531 and Remove the LEVEL column
Batch_Genealogy = Batch_Genealogy[Batch_Genealogy['MOVEMENT_TYPE'] != 531]
Batch_Genealogy = Batch_Genealogy.drop(['BATCH_ID','MATERIAL_ID','LEVEL','TREE_ID','NEW_MATERIAL_DESCRIPTION'], axis=1)

Batch_Genealogy = Batch_Genealogy.drop_duplicates()

QC_Data = QC_Data.drop(['BATCH_ID','MATERIAL_ID'], axis=1)
Campaign_Info = Campaign_Info.drop(['BATCH_ID','MATERIAL_ID'], axis=1)
IoT = IoT.drop(['BATCH_ID','MATERIAL_ID'], axis=1)


3- Add output_id on Batch_Genealogy data

In [32]:
Batch_Genealogy

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,new_id
0,R104490026,2022-02-03,232.100,KG,101,122812P10449
1,R104490027,2022-02-11,253.550,KG,101,122813P10449
2,R104490036,2022-09-16,265.100,KG,101,122825P10449
3,R104490023,2021-11-05,223.900,KG,101,121843P10449
4,R104500007,2021-10-21,227.200,KG,101,121823P10450
...,...,...,...,...,...,...
4187,R104500005,2020-12-29,26.347,KG,261,121905P10010
4189,R104500005,2021-01-05,22.910,KG,261,121906P10010
4191,R104500005,2021-01-06,25.035,KG,261,121907P10010
4193,R104500005,2021-01-13,26.041,KG,261,121908P10010


In [33]:
# Function to get new_id for MOVEMENT_TYPE=101 based on PROCESS_ORDER_NUMBER
def get_output_id(row):
    if row['MOVEMENT_TYPE'] == 261:
        same_order_rows = Batch_Genealogy[(Batch_Genealogy['PROCESS_ORDER_NUMBER'] == row['PROCESS_ORDER_NUMBER']) & (Batch_Genealogy['MOVEMENT_TYPE'] == 101)]
        if len(same_order_rows) > 0:
            return same_order_rows.iloc[0]['new_id']
    elif row['MOVEMENT_TYPE'] == 101:
        return 0
    
    return None

# Apply the function to create the 'output_id' column
Batch_Genealogy['output_id'] = Batch_Genealogy.apply(get_output_id, axis=1)


In [34]:
#Batch_Genealogy.to_excel("C:\\Users\\seyma\\Downloads\\batch.xlsx")

In [35]:
Batch_Genealogy

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,new_id,output_id
0,R104490026,2022-02-03,232.100,KG,101,122812P10449,0
1,R104490027,2022-02-11,253.550,KG,101,122813P10449,0
2,R104490036,2022-09-16,265.100,KG,101,122825P10449,0
3,R104490023,2021-11-05,223.900,KG,101,121843P10449,0
4,R104500007,2021-10-21,227.200,KG,101,121823P10450,0
...,...,...,...,...,...,...,...
4187,R104500005,2020-12-29,26.347,KG,261,121905P10010,121821P10450
4189,R104500005,2021-01-05,22.910,KG,261,121906P10010,121821P10450
4191,R104500005,2021-01-06,25.035,KG,261,121907P10010,121821P10450
4193,R104500005,2021-01-13,26.041,KG,261,121908P10010,121821P10450


Create a Data Frame for Input Materials

In [36]:
output_new_id_list=Batch_Genealogy[Batch_Genealogy["MOVEMENT_TYPE"]==101]["new_id"].unique()
output_materials=Combined_Yields_full["new_id"].unique() #108 materials
common_ids = set(output_materials).intersection(set(output_new_id_list))
print(common_ids)
print("Length of intersection: {}".format(len(common_ids)))

{'122821P10449', '121845P10449', '120934P10446', '120938P10444', '120934P10444', '122826P10449', '121831P10449', '122816P10444', '122823P10449', '122804P10450', '121839P10449', '122788P10443', '122824P10449', '122812P10444', '120938P10449', '122803P10450', '122801P10450', '121820P10450', '122797P10450', '121843P10444', '121829P10449', '121832P10449', '120934P10449', '121844P10449', '120937P10449', '121822P10450', '123711P10444', '120932P10449', '121841P10444', '122827P10449', '122798P10450', '120928P10444', '121835P10449', '120944P10444', '120943P10444', '121840P10444', '120928P10449', '121825P10449', '120935P10449', '122813P10449', '122820P10444', '122825P10449', '121836P10444', '120937P10444', '120936P10449', '120942P10444', '121842P10449', '122814P10444', '121814P10450', '120941P10444', '120928P10446', '122799P10450', '121843P10449', '122822P10449', '122815P10449', '122802P10450', '121834P10449', '122819P10449', '122813P10444', '120935P10444', '122829P10449', '121816P10450', '121837

In [37]:
Batch_Genealogy_input = Batch_Genealogy[Batch_Genealogy['output_id'].isin(common_ids)].copy()

#saving input ids
input_id_list=Batch_Genealogy_input.new_id.unique()

Batch_Genealogy_input

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,new_id,output_id
282,R104440013,2020-04-27,2.000,1,261,113513P20098,120944P10444
283,R104440013,2020-05-05,4.000,1,261,113526P20097,120944P10444
288,R104440024,2020-06-09,24.000,KG,261,113575P20003,121844P10444
290,R104440019,2020-06-09,24.000,KG,261,113575P20003,121836P10444
292,R104440011,2020-04-27,2.000,1,261,113513P20098,120942P10444
...,...,...,...,...,...,...,...
4187,R104500005,2020-12-29,26.347,KG,261,121905P10010,121821P10450
4189,R104500005,2021-01-05,22.910,KG,261,121906P10010,121821P10450
4191,R104500005,2021-01-06,25.035,KG,261,121907P10010,121821P10450
4193,R104500005,2021-01-13,26.041,KG,261,121908P10010,121821P10450


In [55]:
Batch_Genealogy_output = Batch_Genealogy[Batch_Genealogy['new_id'].isin(common_ids)].copy()

#saving output ids
output_id_list=Batch_Genealogy_output.new_id.unique()

Batch_Genealogy_output

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,new_id,output_id
0,R104490026,2022-02-03,232.10,KG,101,122812P10449,0
1,R104490027,2022-02-11,253.55,KG,101,122813P10449,0
2,R104490036,2022-09-16,265.10,KG,101,122825P10449,0
3,R104490023,2021-11-05,223.90,KG,101,121843P10449,0
4,R104500007,2021-10-21,227.20,KG,101,121823P10450,0
...,...,...,...,...,...,...,...
152,R104490011,2021-03-04,233.00,KG,101,121827P10449,0
155,R104490037,2022-10-06,225.70,KG,101,122826P10449,0
176,R104490032,2022-06-23,270.60,KG,101,122821P10449,0
187,R104490031,2022-06-16,236.90,KG,101,122819P10449,0


In [38]:
tree = Batch_Genealogy_input.groupby('output_id')['new_id'].unique()
tree2 = Batch_Genealogy_input.groupby('output_id')['new_id'].nunique()
tree=pd.merge(tree,tree2,on="output_id")
tree=tree.rename(columns={"new_id_x":"input id list","new_id_y":"# of inputs"})

In [39]:
tree

,input id list,# of inputs
output_id,,
120928P10444,"[115685P10004, 115693P10004, 117299P10004, 112...",12
120928P10446,"[112772P20098, 112891P20093, 112892P20095, 112...",12
120928P10449,"[112891P20093, 112892P20095, 112923P20097, 112...",30
120930P10444,"[117299P10004, 112892P20095, 112959P20094, 113...",11
120930P10446,"[112891P20093, 112892P20095, 112923P20097, 112...",12
...,...,...
122826P10449,"[114256P20095, 114277P20003, 114426P20094, 114...",21
122827P10449,"[114256P20095, 114426P20094, 114702P20002, 114...",21
122828P10449,"[114256P20095, 114426P20094, 114721P20093, 114...",21


In [40]:
#tree.to_excel("C:\\Users\\seyma\\Downloads\\output_3.xlsx")

Filtering QC results for input materials

In [81]:
QC_Data_output = QC_Data[QC_Data['new_id'].isin(output_id_list)].copy()
QC_Data_input = QC_Data[QC_Data['new_id'].isin(input_id_list)].copy()
QC_Data_input

,MANUFACTURE_DATE,NEW_QC_TEST,VALUE,new_id
3,2020-03-18,PH,6.9,120932P10010
23,2021-05-19,perte_cre1,NaN,121942P10010
26,2022-05-23,perte_cre1,NaN,123544P10005
34,2021-10-21,lod,6.0,122888P10010
49,2022-06-18,pere_pour_azo,NaN,123274P10004
...,...,...,...,...
428898,2021-11-10,pere_pour_azo,NaN,122895P10010
428903,2021-08-23,cend_sul,NaN,122595P10005
428925,2021-10-13,lla_tel,NaN,121962P10010
428928,2022-06-14,xa_tel,NaN,122973P10010


In [42]:
value_counts = QC_Data_input["new_id"].value_counts().sort_index()

# Display the counts
print(value_counts)

115685P10004    74
115693P10004    74
117268P10004    74
117299P10004    74
119103P10009    74
                ..
123613P10005    74
123614P10005    74
123615P10005    74
123616P10005    74
123617P10005    74
Name: new_id, Length: 787, dtype: int64


Each input has 74 QC test, and output products at least 10 inputs. It is impossible to work with this many features in the model. with lack of information I will eliminate some features. But in real life scenerio, I leverage my domain knowledge to identify the key input products that have a significant impact on the output. I focus on those products that are known to have a strong influence on the manufacturing process or are likely to affect the yield.

number of QC Tests with Value for each input

In [43]:
QC_Data_input[QC_Data_input.VALUE.isna()==False]["new_id"].value_counts().sort_index()


115685P10004    10
115693P10004    10
117268P10004    10
117299P10004    10
119103P10009    10
                ..
123613P10005     6
123614P10005     6
123615P10005     6
123616P10005     6
123617P10005     6
Name: new_id, Length: 787, dtype: int64

In [44]:
# Filter the DataFrame for non-empty 'VALUE' values
QC_Data_input = QC_Data_input[QC_Data_input['VALUE'].notnull()]

# Group the filtered DataFrame by 'new_id' and 'NEW_QC_TEST' and check if all 'new_id' values are present for each 'NEW_QC_TEST'
grouped = QC_Data_input.groupby('new_id')['NEW_QC_TEST'].apply(set)

# Find the intersection of all sets to get the common 'NEW_QC_TEST' values across all 'new_id'
common_qc_tests = set.intersection(*grouped)

# Convert the common_qc_tests set to a list
common_qc_tests_list = list(common_qc_tests)
common_qc_tests_list


['coag_dry', 'PH', 'adn', 'lod']

['adn', 'lod', 'PH', 'coag_dry'] Tests have value and exist for every input

In [45]:
QC_Data_input

,MANUFACTURE_DATE,NEW_QC_TEST,VALUE,new_id
3,2020-03-18,PH,6.900000,120932P10010
34,2021-10-21,lod,6.000000,122888P10010
193,2021-01-12,adn,3.400000,122457P10005
274,2021-01-20,absorbance,0.100000,121844P10010
387,2021-11-08,coag_dry,183.198381,123429P10005
...,...,...,...,...
428342,2021-01-20,absorbance,0.200000,121837P10010
428360,2022-08-11,coag_wet,174.000000,123596P10005
428367,2021-11-22,adn,1.700000,123436P10005
428403,2020-08-24,coag_dry,185.641026,121529P10005


In [46]:
# Create new columns in the DataFrame for each 'NEW_QC_TEST' value and fill them with corresponding 'VALUE' values
for qc_test in common_qc_tests_list:
    QC_Data_input[qc_test] = QC_Data_input.loc[QC_Data_input['NEW_QC_TEST'] == qc_test, 'VALUE']

# Display the updated DataFrame
print(QC_Data_input)


       MANUFACTURE_DATE NEW_QC_TEST       VALUE        new_id    coag_dry  \
3            2020-03-18          PH    6.900000  120932P10010         NaN   
34           2021-10-21         lod    6.000000  122888P10010         NaN   
193          2021-01-12         adn    3.400000  122457P10005         NaN   
274          2021-01-20  absorbance    0.100000  121844P10010         NaN   
387          2021-11-08    coag_dry  183.198381  123429P10005  183.198381   
...                 ...         ...         ...           ...         ...   
428342       2021-01-20  absorbance    0.200000  121837P10010         NaN   
428360       2022-08-11    coag_wet  174.000000  123596P10005         NaN   
428367       2021-11-22         adn    1.700000  123436P10005         NaN   
428403       2020-08-24    coag_dry  185.641026  121529P10005  185.641026   
428450       2022-07-20  absorbance    0.134800  122979P10010         NaN   

         PH  adn  lod  
3       6.9  NaN  NaN  
34      NaN  NaN  6.0  
193

C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\3534371043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_input[qc_test] = QC_Data_input.loc[QC_Data_input['NEW_QC_TEST'] == qc_test, 'VALUE']
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\3534371043.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_input[qc_test] = QC_Data_input.loc[QC_Data_input['NEW_QC_TEST'] == qc_test, 'VALUE']
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\3534371043.py:3: SettingWithCopyWarning: 
A value is 

In [47]:
selected_columns = ['new_id', 'adn', 'lod', 'PH', 'coag_dry']

# Fill NaN values in each column using the 'new_id' column
for column in selected_columns[1:]:
    QC_Data_input[column] = QC_Data_input.groupby('new_id')[column].ffill().bfill()

# Drop duplicates to keep unique rows for each 'new_id'
QC_Data_input = QC_Data_input.drop_duplicates(subset='new_id')

QC_Data_input=QC_Data_input[selected_columns]



C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\2415071066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_input[column] = QC_Data_input.groupby('new_id')[column].ffill().bfill()
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\2415071066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_input[column] = QC_Data_input.groupby('new_id')[column].ffill().bfill()
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\2415071066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [48]:
QC_Data_input

,new_id,adn,lod,PH,coag_dry
3,120932P10010,3.4,6.0,6.90,183.198381
34,122888P10010,3.4,6.0,6.90,183.198381
193,122457P10005,3.4,2.4,6.90,183.198381
274,121844P10010,1.3,2.4,6.90,183.198381
387,123429P10005,1.3,2.4,6.90,183.198381
...,...,...,...,...,...
236566,123438P10005,1.3,3.0,6.80,141.675284
239936,122566P10005,1.1,2.1,6.84,188.465499
239980,123507P10005,1.1,2.1,6.84,170.146138
258612,123496P10005,1.1,1.4,7.00,180.796731


In [49]:
QC_Data_input.isna().sum()

new_id      0
adn         0
lod         0
PH          0
coag_dry    0
dtype: int64

lets try same thing but with empty values. If I have only NEW_QC_TEST name, I need to analyse is it exist or not. So I can create a flag for that. 

In [50]:
# QC_Data_input[QC_Data_input.VALUE.isna()==True]["new_id"].value_counts().sort_index()

# # Filter the DataFrame for non-empty 'VALUE' values
# filtered_df = QC_Data_input[QC_Data_input['VALUE'].isna()==True]

# # Group the filtered DataFrame by 'new_id' and 'NEW_QC_TEST' and check if all 'new_id' values are present for each 'NEW_QC_TEST'
# grouped = filtered_df.groupby('new_id')['NEW_QC_TEST'].apply(set)

# # Find the intersection of all sets to get the common 'NEW_QC_TEST' values across all 'new_id'
# common_qc_tests = set.intersection(*grouped)

# # Convert the common_qc_tests set to a list
# common_qc_tests_list = list(common_qc_tests)
# common_qc_tests_list


Input QC attributes are ready to join! So, Let's find some attributes for Outputs

In [84]:
QC_Data_output

,MANUFACTURE_DATE,NEW_QC_TEST,VALUE,new_id
75,2022-03-25,cend_sul_moy,NaN,122817P10449
76,2020-05-11,perem1cru1,NaN,120928P10446
211,2021-06-21,xa_tel,196.0,121840P10444
327,2022-12-08,pere_pour_lla,1.5,122818P10444
354,2022-11-25,ratio_xa_lla(sec),1.0,122804P10450
...,...,...,...,...
428644,2021-07-08,rende_uimg,NaN,121842P10444
428678,2020-10-12,xa_sec_fourni,NaN,120934P10444
428745,2020-07-13,lod,4.3,120932P10449
428761,2022-02-17,cend_m_creuset1,NaN,122814P10449


In [85]:
# Filter the DataFrame for non-empty 'VALUE' values
QC_Data_output = QC_Data_output[QC_Data_output['VALUE'].notnull()]

# Group the filtered DataFrame by 'new_id' and 'NEW_QC_TEST' and check if all 'new_id' values are present for each 'NEW_QC_TEST'
grouped = QC_Data_output.groupby('new_id')['NEW_QC_TEST'].apply(set)

# Find the intersection of all sets to get the common 'NEW_QC_TEST' values across all 'new_id'
common_qc_tests = set.intersection(*grouped)

# Convert the common_qc_tests set to a list
common_qc_tests_list = list(common_qc_tests)
common_qc_tests_list


['azot_sec',
 'lod',
 'lla_tel',
 'lla_sec',
 'xa_dry',
 'xa_tel',
 'PH',
 'absorbance',
 'ratio_xa_lla(sec)']

In [86]:
# Create new columns in the DataFrame for each 'NEW_QC_TEST' value and fill them with corresponding 'VALUE' values
for qc_test in common_qc_tests_list:
    QC_Data_output[qc_test] = QC_Data_output.loc[QC_Data_output['NEW_QC_TEST'] == qc_test, 'VALUE']

# Display the updated DataFrame
print(QC_Data_output)


       MANUFACTURE_DATE        NEW_QC_TEST       VALUE        new_id  \
211          2021-06-21             xa_tel  196.000000  121840P10444   
327          2022-12-08      pere_pour_lla    1.500000  122818P10444   
354          2022-11-25  ratio_xa_lla(sec)    1.000000  122804P10450   
386          2021-01-29          proteiniq    0.100000  121832P10444   
514          2021-01-29        teneur_chon    0.100000  121832P10444   
...                 ...                ...         ...           ...   
428107       2022-05-12  cend_sul_creuset2   36.241007  122812P10444   
428162       2021-07-16  ratio_xa_lla(sec)    0.983957  121844P10444   
428335       2021-10-07            lla_sec  188.659794  121821P10450   
428624       2022-11-25         absorbance    0.068800  122804P10450   
428745       2020-07-13                lod    4.300000  120932P10449   

        azot_sec  lod  lla_tel     lla_sec  xa_dry  xa_tel  PH  absorbance  \
211          NaN  NaN      NaN         NaN     NaN   196.

C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\3034059552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_output[qc_test] = QC_Data_output.loc[QC_Data_output['NEW_QC_TEST'] == qc_test, 'VALUE']
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\3034059552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_output[qc_test] = QC_Data_output.loc[QC_Data_output['NEW_QC_TEST'] == qc_test, 'VALUE']
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\3034059552.py:3: SettingWithCopyWarning: 
A val

In [87]:
selected_columns = ['new_id','azot_sec',
 'lod',
 'lla_tel',
 'lla_sec',
 'xa_dry',
 'xa_tel',
 'PH',
 'absorbance',
 'ratio_xa_lla(sec)']

# Fill NaN values in each column using the 'new_id' column
for column in selected_columns[1:]:
    QC_Data_output[column] = QC_Data_output.groupby('new_id')[column].ffill().bfill()

# Drop duplicates to keep unique rows for each 'new_id'
QC_Data_output = QC_Data_output.drop_duplicates(subset='new_id')

QC_Data_output=QC_Data_output[selected_columns]



C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\1977962105.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_output[column] = QC_Data_output.groupby('new_id')[column].ffill().bfill()
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\1977962105.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QC_Data_output[column] = QC_Data_output.groupby('new_id')[column].ffill().bfill()
C:\Users\seyma\AppData\Local\Temp\ipykernel_9984\1977962105.py:13: SettingWithCopyWarning: 
A value is trying to be set on

In [88]:
QC_Data_output

,new_id,azot_sec,lod,lla_tel,lla_sec,xa_dry,xa_tel,PH,absorbance,ratio_xa_lla(sec)
211,121840P10444,2.052578,2.5,187.0,189.847716,195.272354,196.0,6.72,0.06066,1.000000
327,122818P10444,2.052578,2.5,187.0,189.847716,195.272354,187.0,6.72,0.06066,1.000000
354,122804P10450,2.052578,2.5,187.0,189.847716,195.272354,187.0,6.72,0.06066,1.000000
386,121832P10444,2.052578,2.5,187.0,189.847716,195.272354,187.0,6.72,0.06066,1.066298
529,122822P10449,2.052578,2.5,187.0,189.847716,195.272354,187.0,6.72,0.06066,1.066298
...,...,...,...,...,...,...,...,...,...,...
74863,121843P10449,2.131902,2.7,192.0,183.013145,190.816327,194.0,6.60,0.06100,1.066298
82381,121845P10449,2.100000,1.9,187.0,188.391039,196.098563,191.0,6.60,0.05620,1.011111
82852,122813P10449,2.106415,2.6,187.0,188.391039,196.098563,191.0,6.60,0.05620,1.011111
83275,121838P10444,2.106415,2.6,187.0,188.391039,196.098563,191.0,6.60,0.05620,1.011111


In [89]:
QC_Data_output.isna().sum()

new_id               0
azot_sec             0
lod                  0
lla_tel              0
lla_sec              0
xa_dry               0
xa_tel               0
PH                   0
absorbance           0
ratio_xa_lla(sec)    0
dtype: int64

QC_Data_input and QC_Data_output are ready! I need to weighted to QC_Data_input data before merge.

4- Merge Dataframes

In [56]:
#Merge YIELD_VALUE on new_id and date (Batch_Genealogy df and Combined_Yields_full df)

merged_df= pd.merge(Batch_Genealogy_output, Combined_Yields_full,on=["MANUFACTURE_DATE","new_id"])



In [58]:
merged_df

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,new_id,output_id,YIELD_VALUE
0,R104490026,2022-02-03,232.10,KG,101,122812P10449,0,1.092185
1,R104490027,2022-02-11,253.55,KG,101,122813P10449,0,1.065421
2,R104490036,2022-09-16,265.10,KG,101,122825P10449,0,1.094054
3,R104490023,2021-11-05,223.90,KG,101,121843P10449,0,1.082375
4,R104500007,2021-10-21,227.20,KG,101,121823P10450,0,1.112719
...,...,...,...,...,...,...,...,...
91,R104490011,2021-03-04,233.00,KG,101,121827P10449,0,1.088523
92,R104490037,2022-10-06,225.70,KG,101,122826P10449,0,1.124211
93,R104490032,2022-06-23,270.60,KG,101,122821P10449,0,1.094243
94,R104490031,2022-06-16,236.90,KG,101,122819P10449,0,1.063160


In [67]:
merged_df_input=pd.merge(Batch_Genealogy_input,QC_Data_input,on="new_id")
merged_df_input

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,new_id,output_id,adn,lod,PH,coag_dry
0,R104440019,2020-12-10,105.040,KG,261,121336P10004,121836P10444,0.8,1.70,6.70,165.803109
1,R104440018,2020-12-10,169.560,KG,261,121336P10004,121834P10444,0.8,1.70,6.70,165.803109
2,R104440002,2018-09-18,60.240,KG,261,117299P10004,120930P10444,0.4,2.20,7.08,181.262729
3,R104440001,2018-09-18,258.630,KG,261,117299P10004,120928P10444,0.4,2.20,7.08,181.262729
4,R104440005,2018-04-04,0.900,KG,261,117268P10004,120935P10444,0.3,2.60,6.90,162.669447
...,...,...,...,...,...,...,...,...,...,...,...
865,R104500005,2021-02-19,26.138,KG,261,121904P10010,121821P10450,0.4,3.38,6.76,181.352459
866,R104500005,2020-12-29,26.347,KG,261,121905P10010,121821P10450,1.2,2.20,7.00,178.053830
867,R104500005,2021-01-05,22.910,KG,261,121906P10010,121821P10450,0.4,1.40,7.00,162.886598
868,R104500005,2021-01-06,25.035,KG,261,121907P10010,121821P10450,0.2,2.80,6.80,160.123967


Weighted QC attributes = ∑_(i=1)^n▒Qi⋅QCi/(∑_ ^ ▒Qi)

In [76]:
output_quantity_sum = merged_df_input.groupby('output_id')['QUANTITY'].sum()

merged_df_input = merged_df_input.merge(output_quantity_sum, on='output_id', suffixes=['', '_sum'])

merged_df_input['adn_weighted'] = merged_df_input['adn'] * merged_df_input['QUANTITY'] / merged_df_input['QUANTITY_sum']
merged_df_input['lod_weighted'] = merged_df_input['lod'] * merged_df_input['QUANTITY'] / merged_df_input['QUANTITY_sum']
merged_df_input['PH_weighted'] = merged_df_input['PH'] * merged_df_input['QUANTITY'] / merged_df_input['QUANTITY_sum']
merged_df_input['coag_dry_weighted'] = merged_df_input['coag_dry'] * merged_df_input['QUANTITY'] / merged_df_input['QUANTITY_sum']

output_weighted = merged_df_input.groupby('output_id')[['adn_weighted', 'lod_weighted', 'PH_weighted', 'coag_dry_weighted']].sum().reset_index()


In [77]:
output_weighted

,output_id,adn_weighted,lod_weighted,PH_weighted,coag_dry_weighted
0,120928P10444,0.729567,2.147323,7.039030,181.146456
1,120928P10446,1.477580,4.176253,7.381867,173.331745
2,120928P10449,1.177585,2.172667,6.905130,175.465691
3,120930P10444,1.199200,1.640560,6.776304,168.759820
4,120930P10446,1.188480,2.818023,6.925393,174.520499
...,...,...,...,...,...
91,122826P10449,1.260885,2.308404,6.957182,172.831156
92,122827P10449,1.463716,2.277895,6.919283,174.993016
93,122828P10449,0.795234,2.136022,6.883880,172.391303
94,122829P10449,1.118410,2.576637,6.908546,175.356745


Merge output QC tests and Batch_Genealogy Table

In [92]:
merged_df=pd.merge(merged_df, QC_Data_output,on="new_id")
merged_df

,PROCESS_ORDER_NUMBER,MANUFACTURE_DATE,QUANTITY,UNIT_OF_MEASURE,MOVEMENT_TYPE,new_id,output_id,YIELD_VALUE,azot_sec,lod,lla_tel,lla_sec,xa_dry,xa_tel,PH,absorbance,ratio_xa_lla(sec)
0,R104490026,2022-02-03,232.10,KG,101,122812P10449,0,1.092185,2.052578,2.6,183.0,190.721649,196.098563,196.0,6.90,0.03300,1.005155
1,R104490027,2022-02-11,253.55,KG,101,122813P10449,0,1.065421,2.106415,2.6,187.0,188.391039,196.098563,191.0,6.60,0.05620,1.011111
2,R104490036,2022-09-16,265.10,KG,101,122825P10449,0,1.094054,2.100000,3.2,183.0,190.721649,196.098563,196.0,6.90,0.03300,1.005155
3,R104490023,2021-11-05,223.90,KG,101,121843P10449,0,1.082375,2.131902,2.7,192.0,183.013145,190.816327,194.0,6.60,0.06100,1.066298
4,R104500007,2021-10-21,227.20,KG,101,121823P10450,0,1.112719,2.131902,2.3,183.0,186.544343,196.098563,183.0,6.72,0.05580,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,R104490011,2021-03-04,233.00,KG,101,121827P10449,0,1.088523,2.052578,2.5,183.0,190.721649,192.783505,192.0,6.87,0.06892,1.066298
92,R104490037,2022-10-06,225.70,KG,101,122826P10449,0,1.124211,2.052578,2.5,187.0,189.847716,195.272354,187.0,6.72,0.06066,1.066298
93,R104490032,2022-06-23,270.60,KG,101,122821P10449,0,1.094243,2.052578,2.6,187.0,184.917355,192.229039,192.0,6.87,0.06100,1.005102
94,R104490031,2022-06-16,236.90,KG,101,122819P10449,0,1.063160,2.052578,2.5,187.0,184.917355,192.229039,192.0,6.87,0.06100,1.066298


In [94]:
#I will change new_id column into output_id column and drop output_id=0 column
merged_df=merged_df.drop(['PROCESS_ORDER_NUMBER', 'MANUFACTURE_DATE','UNIT_OF_MEASURE', 'MOVEMENT_TYPE','output_id'],axis=1)

In [96]:
merged_df.rename(columns={'new_id': 'output_id'}, inplace=True)


In [98]:
merged_df.columns

Index(['QUANTITY', 'output_id', 'YIELD_VALUE', 'azot_sec', 'lod', 'lla_tel',
       'lla_sec', 'xa_dry', 'xa_tel', 'PH', 'absorbance', 'ratio_xa_lla(sec)'],
      dtype='object')

In [100]:
merged_df=merged_df[['output_id', 'QUANTITY',  'azot_sec', 'lod', 'lla_tel','lla_sec', 'xa_dry', 'xa_tel', 'PH', 'absorbance', 'ratio_xa_lla(sec)','YIELD_VALUE']]

In [101]:
merged_df

,output_id,QUANTITY,azot_sec,lod,lla_tel,lla_sec,xa_dry,xa_tel,PH,absorbance,ratio_xa_lla(sec),YIELD_VALUE
0,122812P10449,232.10,2.052578,2.6,183.0,190.721649,196.098563,196.0,6.90,0.03300,1.005155,1.092185
1,122813P10449,253.55,2.106415,2.6,187.0,188.391039,196.098563,191.0,6.60,0.05620,1.011111,1.065421
2,122825P10449,265.10,2.100000,3.2,183.0,190.721649,196.098563,196.0,6.90,0.03300,1.005155,1.094054
3,121843P10449,223.90,2.131902,2.7,192.0,183.013145,190.816327,194.0,6.60,0.06100,1.066298,1.082375
4,121823P10450,227.20,2.131902,2.3,183.0,186.544343,196.098563,183.0,6.72,0.05580,1.000000,1.112719
...,...,...,...,...,...,...,...,...,...,...,...,...
91,121827P10449,233.00,2.052578,2.5,183.0,190.721649,192.783505,192.0,6.87,0.06892,1.066298,1.088523
92,122826P10449,225.70,2.052578,2.5,187.0,189.847716,195.272354,187.0,6.72,0.06066,1.066298,1.124211
93,122821P10449,270.60,2.052578,2.6,187.0,184.917355,192.229039,192.0,6.87,0.06100,1.005102,1.094243
94,122819P10449,236.90,2.052578,2.5,187.0,184.917355,192.229039,192.0,6.87,0.06100,1.066298,1.063160


And Finally, merged with input QC attributes

In [102]:
merged_df=pd.merge(merged_df, output_weighted,on="output_id")

In [104]:
merged_df.columns

Index(['output_id', 'QUANTITY', 'azot_sec', 'lod', 'lla_tel', 'lla_sec',
       'xa_dry', 'xa_tel', 'PH', 'absorbance', 'ratio_xa_lla(sec)',
       'YIELD_VALUE', 'adn_weighted', 'lod_weighted', 'PH_weighted',
       'coag_dry_weighted'],
      dtype='object')

In [105]:
merged_df=merged_df[['output_id', 'QUANTITY', 'azot_sec', 'lod', 'lla_tel', 'lla_sec',
       'xa_dry', 'xa_tel', 'PH', 'absorbance', 'ratio_xa_lla(sec)', 'adn_weighted', 'lod_weighted', 'PH_weighted',
       'coag_dry_weighted',
       'YIELD_VALUE']]

In [106]:
merged_df

,output_id,QUANTITY,azot_sec,lod,lla_tel,lla_sec,xa_dry,xa_tel,PH,absorbance,ratio_xa_lla(sec),adn_weighted,lod_weighted,PH_weighted,coag_dry_weighted,YIELD_VALUE
0,122812P10449,232.10,2.052578,2.6,183.0,190.721649,196.098563,196.0,6.90,0.03300,1.005155,1.590833,1.998518,6.918765,180.029381,1.092185
1,122813P10449,253.55,2.106415,2.6,187.0,188.391039,196.098563,191.0,6.60,0.05620,1.011111,1.057432,3.132995,6.959613,168.953098,1.065421
2,122825P10449,265.10,2.100000,3.2,183.0,190.721649,196.098563,196.0,6.90,0.03300,1.005155,0.971856,2.547256,6.912524,177.366063,1.094054
3,121843P10449,223.90,2.131902,2.7,192.0,183.013145,190.816327,194.0,6.60,0.06100,1.066298,0.742307,2.264157,6.935205,178.335101,1.082375
4,121823P10450,227.20,2.131902,2.3,183.0,186.544343,196.098563,183.0,6.72,0.05580,1.000000,1.167632,2.579613,6.939433,175.203956,1.112719
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,121827P10449,233.00,2.052578,2.5,183.0,190.721649,192.783505,192.0,6.87,0.06892,1.066298,1.411217,2.310562,6.911807,174.369532,1.088523
92,122826P10449,225.70,2.052578,2.5,187.0,189.847716,195.272354,187.0,6.72,0.06066,1.066298,1.260885,2.308404,6.957182,172.831156,1.124211
93,122821P10449,270.60,2.052578,2.6,187.0,184.917355,192.229039,192.0,6.87,0.06100,1.005102,0.948015,2.617961,6.920396,166.431553,1.094243
94,122819P10449,236.90,2.052578,2.5,187.0,184.917355,192.229039,192.0,6.87,0.06100,1.066298,1.282221,2.007113,6.911070,175.074612,1.063160


5- Standardization

6- Feature Regularization

7- Save the Data